In [6]:
import numpy as np
import ast
import nltk
import pickle
from sklearn.model_selection import train_test_split


with open('dataset/nap_no_weight.json', 'r') as f:
    content = f.readlines()

nap = []
for c in content:
    nap.append(ast.literal_eval(c))

with open('dataset/vocabularies.json') as f:
    vocab = f.readlines()
    vocab = ast.literal_eval(vocab[0])


def encode(editor_note):
    res = np.zeros(len(vocab), dtype=float)
    for i, c in enumerate(vocab):
        if c.lower() in editor_note.lower():
            res[i] = 0.99
        else:
            res[i] = 0.01
    return res

In [24]:
raw_data = []
for n in nap:
    editor_note = n['editor_note']
    raw_data.append(editor_note.lower().replace('\n', '').split('shown here with')[0].replace('exclusive at net-a-porter.com.', '').strip()])
    #raw_data.append([editor_note, encode(editor_note)])

In [26]:
for i, d in enumerate(raw_data):
    raw_data[i].append(encode(d[0]))

In [28]:
with open("raw_dataset.p", "wb") as pickle_d:
    pickle.dump(raw_data, pickle_d)

In [ ]:
with open("raw_dataset.p", "wb") as pickle_d:
    pickle.dump(raw_data, pickle_d)

with open("raw_dataset.p", "rb") as pickle_d:
    raw_data = pickle.load(pickle_d)

def get_pos(sentence):
    tokens = nltk.word_tokenize(sentence)
    res =  nltk.pos_tag(tokens)
    res.append(('<END>', 'END'))
    res.insert(0, ('<START>', 'START'))
    return res

filtered_data = []
for i in range(len(raw_data)):
    raw_data[i][0] = raw_data[i][0].replace('\n', '').replace('\'s', ' \'s')
    if np.sum(raw_data[i][1]) * 1.0 / len(raw_data[i][0].split(' ')) > 0.2 and np.sum(raw_data[i][1]) > 15:
        raw_data[i].append(get_pos(raw_data[i][0]))
        filtered_data.append(raw_data[i])

train_dataset, test_dataset = train_test_split(filtered_data, test_size=0.2, random_state=42)

with open("train_dataset.p", "wb") as pickle_d:
    pickle.dump(train_dataset, pickle_d)

with open("test_dataset.p", "wb") as pickle_d:
    pickle.dump(test_dataset, pickle_d)



In [32]:
from keras.preprocessing import sequence
sequence.pad_sequences([[312,321,325,54]], maxlen=3, padding='post')

array([[321, 325,  54]], dtype=int32)

In [35]:
with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [63]:
for i, v in enumerate(raw_data):
    raw_data[i][0] = nltk.word_tokenize(raw_data[i][0])

In [64]:
import gensim

In [67]:
word_embedding_input = []
for d in raw_data:
    word_embedding_input.append(d[0])

In [69]:
model = gensim.models.Word2Vec (word_embedding_input, size=100, window=10, min_count=2, workers=10)

NameError: name 'documents' is not defined

In [70]:
model.train(word_embedding_input,total_examples=len(word_embedding_input),epochs=10)

(24092576, 33199720)

In [72]:
w1 = "small"
model.wv.most_similar (positive=w1)

/Users/yuanshen/miniconda3/envs/py365/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('large', 0.6473848819732666),
 ('twill-lined', 0.6215367913246155),
 ('larger', 0.6164891719818115),
 ('compact', 0.6114650964736938),
 ('smaller', 0.600143313407898),
 ('canvas-lined', 0.582280158996582),
 ('protective', 0.57435142993927),
 ('purse', 0.5608631372451782),
 ('compartmentalized', 0.5601623058319092),
 ('laptop', 0.5563493967056274)]

In [73]:
model.save('word_embedding.model')

In [75]:
len(model.wv['large'])

100

In [76]:
with open("raw_dataset.p", "rb") as pickle_d:
    raw_data = pickle.load(pickle_d)

In [77]:
raw_data[0]

["nike's 'power speed' leggings are designed with intense running in mind, from uphill sprints to long-distance marathon training. made from sweat-wicking dri-fit fabric, they have a close, compressive fit that supports key muscles as you move.",
 array([0.99, 0.99, 0.01, ..., 0.01, 0.01, 0.01])]

In [85]:
import re
s = "string. With. Punctuation?"
s = re.sub(r'[^\P{P}\-.]+','',s)

error: bad escape \P at position 2

In [81]:
s

'string With Punctuation'

In [87]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [90]:
len(embeddings_index['treeloved'])

KeyError: 'treeloved'